02 파일과 feature 동일, autogluon 사용

**AutoGluon TabularPredictor**
- `time_limit`: 7200초 (2시간)
- `presets`: 'best_quality'
- `num_bag_folds`: 5-Fold CV
- `num_stack_levels`: 1 (Multi-layer Stacking)


In [1]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("AutoGluon Experiment")
print("="*60)

# 1. 데이터 로드
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")
submission = pd.read_csv("../data/sample_submission.csv")

print(f"\nTrain shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

# 2. 전처리 함수 (기존과 동일)
def preprocess(df):
    df_copy = df.copy()
    
    # 시술_대분류
    def major_procedure(x):
        if pd.isna(x):
            return "Unknown"
        if "IUI" in x:
            return "IUI"
        if "DI" in x:
            return "Other"
        if "ICSI" in x:
            return "ICSI"
        if "IVF" in x:
            return "IVF"
        return "Other"
    
    df_copy["시술_대분류"] = df_copy["특정 시술 유형"].apply(major_procedure)
    
    # BLASTOCYST 포함 여부
    df_copy["BLASTOCYST_포함"] = df_copy["특정 시술 유형"].str.contains("BLASTOCYST", na=False).astype(int)
    
    # 배아 이식 여부
    embryo_stage_cols = [
        "단일 배아 이식 여부", "착상 전 유전 진단 사용 여부", "배아 생성 주요 이유",
        "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수",
        "이식된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수",
        "미세주입 후 저장된 배아 수", "해동된 배아 수", "해동 난자 수",
        "수집된 신선 난자 수", "저장된 신선 난자 수", "혼합된 난자 수",
        "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수",
        "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부",
    ]
    
    df_copy["배아_이식_미도달"] = df_copy[embryo_stage_cols].isna().all(axis=1).astype(int)
    df_copy["배아_이식_여부"] = 1 - df_copy["배아_이식_미도달"]
    
    # 배아 진행 단계
    def embryo_stage(row):
        if row['배아_이식_여부'] == 0:
            return '배아단계_미도달'
        elif pd.isna(row['총 생성 배아 수']) or row['총 생성 배아 수'] == 0:
            return '배아생성_실패'
        elif pd.isna(row['이식된 배아 수']) or row['이식된 배아 수'] == 0:
            return '이식_미실시'
        else:
            return '이식_완료'
    
    df_copy['배아_진행_단계'] = df_copy.apply(embryo_stage, axis=1)
    
    # 총시술_bin3
    def collapse_trials(x):
        if x == '0회':
            return '0회'
        elif x in ['1회', '2회']:
            return '1–2회'
        else:
            return '3회 이상'
    
    df_copy["총시술_bin3"] = df_copy["총 시술 횟수"].apply(collapse_trials)
    
    # 나이_3구간
    def age_group_simple(age):
        if age == '알 수 없음':
            return 'Unknown'
        elif age == '만18-34세':
            return '34세 이하'
        elif age in ['만35-37세', '만38-39세']:
            return '35-39세'
        else:
            return '40세 이상'
    
    df_copy['나이_3구간'] = df_copy['시술 당시 나이'].apply(age_group_simple)
    
    # 이식배아_구간
    def embryo_count_bin(count):
        if pd.isna(count) or count == 0:
            return '0개'
        elif count <= 2:
            return '1-2개'
        else:
            return '3개 이상'
    
    df_copy['이식배아_구간'] = df_copy['이식된 배아 수'].apply(embryo_count_bin)
    
    # Day5_이식_여부
    df_copy['Day5_이식_여부'] = (df_copy['배아 이식 경과일'] == 5.0).astype(int)
    
    # 불임원인_복잡도
    infertility_cols = [
        "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인",
        "부부 주 불임 원인", "부부 부 불임 원인", "불명확 불임 원인",
        "불임 원인 - 난관 질환", "불임 원인 - 남성 요인", "불임 원인 - 배란 장애",
        "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
        "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태"
    ]
    
    df_copy["불임_원인_개수"] = df_copy[infertility_cols].sum(axis=1)
    
    def infertility_complexity(count):
        if count == 0:
            return 'None'
        elif count == 1:
            return 'Single'
        elif count == 2:
            return 'Double'
        else:
            return 'Multiple'
    
    df_copy['불임원인_복잡도'] = df_copy['불임_원인_개수'].apply(infertility_complexity)
    
    # 배아_해동_실시_여부
    df_copy['배아_해동_실시_여부'] = df_copy['배아 해동 경과일'].notna().astype(int)
    
    # 배아 효율 비율 변수들
    df_copy['배아_생성_효율'] = df_copy['총 생성 배아 수'] / (df_copy['수집된 신선 난자 수'] + 1)
    df_copy['배아_이식_비율'] = df_copy['이식된 배아 수'] / (df_copy['총 생성 배아 수'] + 1)
    df_copy['배아_저장_비율'] = df_copy['저장된 배아 수'] / (df_copy['총 생성 배아 수'] + 1)
    
    # 교호작용 변수들
    df_copy['나이×Day5'] = df_copy['시술 당시 나이'].astype(str) + '_' + df_copy['Day5_이식_여부'].astype(str)
    df_copy['시술횟수×나이'] = df_copy['총시술_bin3'] + '_' + df_copy['나이_3구간']
    
    return df_copy

# 3. 전처리 적용
print("\n전처리 중...")
train_df = preprocess(train_df)
test_df = preprocess(test_df)

print(f"전처리 후 Train shape: {train_df.shape}")
print(f"전처리 후 Test shape: {test_df.shape}")

# 4. ID 제거 (AutoGluon은 자동으로 타겟 찾음)
train_data = train_df.drop(columns=['ID', '배아_이식_미도달'])
test_data = test_df.drop(columns=['ID', '배아_이식_미도달'])

print(f"\nFinal Train shape: {train_data.shape}")
print(f"Target distribution:\n{train_data['임신 성공 여부'].value_counts(normalize=True)}")

# ============================
# 5. AutoGluon 학습
# ============================
print("\n" + "="*60)
print("Starting AutoGluon Training")
print("="*60)
print("\n⏰ 예상 소요 시간: 2시간")
print("💾 메모리 사용량: 4-8GB 예상")
print("\n학습 시작...\n")

predictor = TabularPredictor(
    label='임신 성공 여부',
    eval_metric='roc_auc',
    problem_type='binary',
    path='../outputs/autogluon_models'
)

predictor.fit(
    train_data=train_data,
    time_limit=7200,  # 2시간
    presets='best_quality',
    num_bag_folds=5,
    num_bag_sets=1,
    num_stack_levels=1,
    verbosity=2
)

# ============================
# 6. 결과 확인
# ============================
print("\n" + "="*60)
print("AutoGluon Results")
print("="*60)

# Leaderboard
leaderboard = predictor.leaderboard(train_data, silent=True)
print("\n=== Model Leaderboard (Top 15) ===")
print(leaderboard.head(15).to_string())

# Best model
best_model = leaderboard.iloc[0]['model']
best_score = leaderboard.iloc[0]['score_val']
print(f"\n🏆 Best Model: {best_model}")
print(f"🎯 Best CV AUC: {best_score:.6f}")

# ============================
# 7. Feature Importance
# ============================
print("\n" + "="*60)
print("Feature Importance")
print("="*60)

try:
    importance = predictor.feature_importance(train_data)
    print("\n=== Top 20 Features ===")
    print(importance.head(20).to_string())
except:
    print("Feature importance not available for ensemble model")

# ============================
# 8. 예측 및 Submission
# ============================
print("\n" + "="*60)
print("Generating Predictions")
print("="*60)

# Test 예측
test_predictions = predictor.predict_proba(test_data)

# 확률값 추출 (클래스 1의 확률)
if isinstance(test_predictions, pd.DataFrame):
    test_proba = test_predictions[1].values
else:
    test_proba = test_predictions

print(f"\n예측값 통계:")
print(f"Min:  {test_proba.min():.6f}")
print(f"Max:  {test_proba.max():.6f}")
print(f"Mean: {test_proba.mean():.6f}")
print(f"Std:  {test_proba.std():.6f}")

# Submission 생성
submission['probability'] = test_proba
submission.to_csv("../outputs/submission_autogluon(0206).csv", index=False)

print("\n✅ AutoGluon Submission saved!")
print(f"   File: ../outputs/submission_autogluon(0206).csv")

# ============================
# 9. 상세 정보 저장
# ============================
print("\n" + "="*60)
print("Saving Details")
print("="*60)

# Leaderboard 전체 저장
leaderboard.to_csv("../outputs/autogluon_leaderboard.csv", index=False)
print("✅ Leaderboard saved: ../outputs/autogluon_leaderboard.csv")

# Feature importance 저장
try:
    importance.to_csv("../outputs/autogluon_feature_importance.csv")
    print("✅ Feature importance saved: ../outputs/autogluon_feature_importance.csv")
except:
    pass

# ============================
# 10. 요약
# ============================
print("\n" + "="*60)
print("SUMMARY")
print("="*60)

summary = f"""
Best Model: {best_model}
CV AUC: {best_score:.6f}

Total Models Trained: {len(leaderboard)}
Ensemble Type: {predictor.model_best}

Top 3 Models:
"""

for i in range(min(3, len(leaderboard))):
    summary += f"  {i+1}. {leaderboard.iloc[i]['model']}: {leaderboard.iloc[i]['score_val']:.6f}\n"

print(summary)

# 요약 파일 저장
with open("../outputs/autogluon_summary.txt", "w") as f:
    f.write(summary)

print("\n✅ Summary saved: ../outputs/autogluon_summary.txt")
print("\n" + "="*60)
print("AutoGluon Experiment Complete! 🎉")
print("="*60)

/opt/anaconda3/envs/fertility_ai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AutoGluon Experiment

Train shape: (256351, 69)
Test shape: (90067, 68)

전처리 중...


Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=5, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 7200 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: ../outputs/autogluon_models/ds_sub_fit/sub_fit_ho.


전처리 후 Train shape: (256351, 86)
전처리 후 Test shape: (90067, 85)

Final Train shape: (256351, 84)
Target distribution:
임신 성공 여부
0    0.741651
1    0.258349
Name: proportion, dtype: float64

Starting AutoGluon Training

⏰ 예상 소요 시간: 2시간
💾 메모리 사용량: 4-8GB 예상

학습 시작...



2026-02-06 11:20:02,405	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Running the sub-fit in a ray process to avoid memory leakage.
Spend 1849 seconds for the sub-fit(s) during dynamic stacking.
Time left for full fit of AutoGluon: 5351 seconds.
Starting full fit now with num_stack_levels 0.
Beginning AutoGluon training ... Time limit = 5351s
AutoGluon will save models to "../outputs/autogluon_models"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.6.0: Mon Jul 14 11:30:29 PDT 2025; root:xnu-11417.140.69~1/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       4.73 GB / 16.00 GB (29.6%)
Disk Space Avail:   227.67 GB / 460.43 GB (49.4%)
Train Data Rows:    256351
Train Data Columns: 83
Label Column:       임신 성공 여부
Problem Type:


AutoGluon Results


These features in provided data are not utilized by the predictor and will be ignored: ['불임 원인 - 여성 요인', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아_이식_여부']
Computing feature importance via permutation shuffling for 77 features using 5000 rows with 5 shuffle sets...



=== Model Leaderboard (Top 15) ===
                      model  score_test  score_val eval_metric  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0     ExtraTrees_r42_BAG_L1    0.958349   0.722289     roc_auc        2.102997       8.557869    59.288020                 2.102997                8.557869          59.288020            1       True         22
1   RandomForestEntr_BAG_L1    0.930159   0.731145     roc_auc        2.477784       8.096630    13.980618                 2.477784                8.096630          13.980618            1       True          6
2   RandomForestGini_BAG_L1    0.928349   0.730036     roc_auc        2.458589       7.909825    13.528742                 2.458589                7.909825          13.528742            1       True          5
3     ExtraTreesEntr_BAG_L1    0.903508   0.732631     roc_auc        2.593283       7.924929    11.344786                 2

	853.67s	= Expected runtime (170.73s per shuffle set)
	544.7s	= Actual runtime (Completed 5 of 5 shuffle sets)



=== Top 20 Features ===
                  importance    stddev       p_value  n  p99_high   p99_low
이식된 배아 수            0.019851  0.004014  1.901256e-04  5  0.028117  0.011586
시술 당시 나이            0.011297  0.003085  6.059486e-04  5  0.017649  0.004945
나이_3구간              0.010647  0.003570  1.313894e-03  5  0.017998  0.003296
배아_이식_비율            0.009838  0.002147  2.555314e-04  5  0.014258  0.005418
이식배아_구간             0.009680  0.003194  1.237507e-03  5  0.016256  0.003103
나이×Day5             0.008002  0.002413  8.827383e-04  5  0.012971  0.003033
배아_저장_비율            0.007536  0.002599  1.459067e-03  5  0.012888  0.002184
배아 이식 경과일           0.007275  0.001796  4.120106e-04  5  0.010974  0.003576
총 생성 배아 수           0.006966  0.002667  2.142094e-03  5  0.012457  0.001475
배아_생성_효율            0.006736  0.001112  8.599298e-05  5  0.009026  0.004446
시술 시기 코드            0.006650  0.000500  3.817704e-06  5  0.007680  0.005620
배아_진행_단계            0.004722  0.001765  1.961776e-03  5  0.0083

In [2]:
print("Submission model:", predictor.model_best)

Submission model: WeightedEnsemble_L2


AUC 기준 WeightedEnsemble_L2 1등 0.740718

------------

<계속해서 상위권에 있는 원본 변수>

이식된 배아 수 (0.0199) 

시술 당시 나이 (0.0113)

배아 이식 경과일 (0.0073)

총 생성 배아 수 (0.0067)

저장된 배아 수

-------------

<파생 변수>
배아_이식_비율 

나이×Day5

배아_저장_비율

배아_생성_효율



## 03.1 feature top 20만 가지고 학습

In [3]:
import os
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("03.1 - Top 20 Feature Selection + AutoGluon")
print("="*80)
print("\n전략: 03번 Feature Importance Top 20만 사용")
print("="*80)

# ============================================================
# 0) Load
# ============================================================
TRAIN_PATH = "../data/train.csv"
TEST_PATH  = "../data/test.csv"
SUB_PATH   = "../data/sample_submission.csv"

train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sub   = pd.read_csv(SUB_PATH)

TARGET_COL = "임신 성공 여부"
ID_COL = "ID"

print(f"\nTrain shape: {train.shape}")
print(f"Test shape: {test.shape}")

# ============================================================
# 1) Feature Engineering (03번과 동일)
# ============================================================

# 고결측 변수 제거
HIGH_MISSING_COLS = [
    '난자 해동 경과일',           # 99.4%
    'PGS 시술 여부',             # 99.2%
    'PGD 시술 여부',             # 99.1%
    '착상 전 유전 검사 사용 여부', # 98.9%
    '임신 시도 또는 마지막 임신 경과 연수',  # 96.3%
]

EMBRYO_STAGE_COLS = [
    "단일 배아 이식 여부","착상 전 유전 진단 사용 여부","배아 생성 주요 이유",
    "총 생성 배아 수","미세주입된 난자 수","미세주입에서 생성된 배아 수",
    "이식된 배아 수","미세주입 배아 이식 수","저장된 배아 수",
    "미세주입 후 저장된 배아 수","해동된 배아 수","해동 난자 수",
    "수집된 신선 난자 수","저장된 신선 난자 수","혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수","기증자 정자와 혼합된 난자 수",
    "동결 배아 사용 여부","신선 배아 사용 여부","기증 배아 사용 여부","대리모 여부",
]

INFERTILITY_COLS = [
    "남성 주 불임 원인","남성 부 불임 원인","여성 주 불임 원인","여성 부 불임 원인",
    "부부 주 불임 원인","부부 부 불임 원인","불명확 불임 원인",
    "불임 원인 - 난관 질환","불임 원인 - 남성 요인","불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인","불임 원인 - 자궁경부 문제","불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도","불임 원인 - 정자 면역학적 요인","불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태"
]

def preprocess(df):
    df = df.copy()
    
    # 고결측 제거
    df = df.drop(columns=HIGH_MISSING_COLS, errors='ignore')
    
    # 시술_대분류
    def major_procedure(x):
        if pd.isna(x): return "Unknown"
        x = str(x)
        if "IUI" in x: return "IUI"
        if "ICSI" in x: return "ICSI"
        if "IVF" in x: return "IVF"
        if "DI" in x: return "DI"
        return "Other"
    
    df["시술_대분류"] = df["특정 시술 유형"].apply(major_procedure)
    
    # BLASTOCYST_포함
    s = df["특정 시술 유형"].astype("object").fillna("Unknown").astype(str)
    df["BLASTOCYST_포함"] = s.str.contains("BLASTOCYST", na=False).astype(int)
    
    # 배아_이식_여부
    df["배아_stage_missing_count"] = df[EMBRYO_STAGE_COLS].isna().sum(axis=1)
    df["배아_이식_미도달"] = (df["배아_stage_missing_count"] == len(EMBRYO_STAGE_COLS)).astype(int)
    df["배아_이식_여부"] = 1 - df["배아_이식_미도달"]
    
    # 배아_진행_단계
    def embryo_stage(row):
        if row['배아_이식_여부'] == 0:
            return '배아단계_미도달'
        total = pd.to_numeric(row.get('총 생성 배아 수'), errors='coerce')
        implanted = pd.to_numeric(row.get('이식된 배아 수'), errors='coerce')
        if pd.isna(total) or total == 0:
            return '배아생성_실패'
        elif pd.isna(implanted) or implanted == 0:
            return '이식_미실시'
        else:
            return '이식_완료'
    
    df['배아_진행_단계'] = df.apply(embryo_stage, axis=1)
    
    # 총시술_bin3
    def collapse_trials(x):
        if pd.isna(x): return "Unknown"
        if x == "0회": return "0회"
        if x in ["1회", "2회"]: return "1-2회"
        return "3회 이상"
    
    df["총시술_bin3"] = df["총 시술 횟수"].apply(collapse_trials)
    
    # 나이_3구간
    def age_group_simple(age):
        if pd.isna(age) or age == "알 수 없음": return "Unknown"
        if age == "만18-34세": return "34세 이하"
        if age in ["만35-37세", "만38-39세"]: return "35-39세"
        return "40세 이상"
    
    df["나이_3구간"] = df["시술 당시 나이"].apply(age_group_simple)
    
    # 이식배아_구간
    def embryo_count_bin(count):
        count = pd.to_numeric(count, errors='coerce')
        if pd.isna(count) or count == 0: return "0개"
        elif count <= 2: return "1-2개"
        else: return "3개 이상"
    
    df['이식배아_구간'] = df['이식된 배아 수'].apply(embryo_count_bin)
    
    # Day5_이식_여부
    d = pd.to_numeric(df["배아 이식 경과일"], errors="coerce")
    df["Day5_이식_여부"] = (d == 5).astype(int)
    
    # 불임원인_복잡도
    tmp = df[INFERTILITY_COLS].apply(pd.to_numeric, errors="coerce").fillna(0)
    df["불임_원인_개수"] = tmp.sum(axis=1)
    
    def infertility_complexity(count):
        if count == 0: return 'None'
        elif count == 1: return 'Single'
        elif count == 2: return 'Double'
        else: return 'Multiple'
    
    df['불임원인_복잡도'] = df['불임_원인_개수'].apply(infertility_complexity)
    
    # 배아_해동_실시_여부
    df['배아_해동_실시_여부'] = df['배아 해동 경과일'].notna().astype(int)
    
    # 숫자형 안전 변환
    num_cols = [
        "총 생성 배아 수", "이식된 배아 수", "저장된 배아 수",
        "수집된 신선 난자 수"
    ]
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    
    # 비율 변수
    df['배아_생성_효율'] = df['총 생성 배아 수'] / (df['수집된 신선 난자 수'] + 1)
    df['배아_이식_비율'] = df['이식된 배아 수'] / (df['총 생성 배아 수'] + 1)
    df['배아_저장_비율'] = df['저장된 배아 수'] / (df['총 생성 배아 수'] + 1)
    
    # 교호작용
    df['나이×Day5'] = df['시술 당시 나이'].astype(str) + '_' + df['Day5_이식_여부'].astype(str)
    df['시술횟수×나이'] = df['총시술_bin3'].astype(str) + '_' + df['나이_3구간'].astype(str)
    
    return df

print("\n전처리 중...")
train_df = preprocess(train)
test_df = preprocess(test)

print(f"✅ 전처리 완료")
print(f"   Train: {train_df.shape}")
print(f"   Test: {test_df.shape}")

# ============================================================
# 🎯 2) Top 20 Feature 선택 (핵심!)
# ============================================================
print("\n" + "="*80)
print("🎯 Top 20 Feature Selection (03번 Feature Importance 기반)")
print("="*80)

# 03번 Feature Importance Top 20
TOP_20_FEATURES = [
    # Top 10 (원본)
    '이식된 배아 수',           # 1위 (0.0300)
    '시술 당시 나이',           # 2위 (0.0112)
    '나이_3구간',              # 3위 (0.0106)
    '배아_저장_비율',           # 4위 (0.0097)
    '이식배아_구간',           # 5위 (0.0097)
    '나이×Day5',             # 7위 (0.0090)
    'Day5_이식_여부',         # 8위 (0.0067)
    '배아_생성_효율',          # 9위 (0.0067)
    '시술 시기 코드',          # 10위 (0.0065)
    '저장된 배아 수',          # 11위 (0.0064)
    
    # Top 11-20
    '배아_이식_비율',          # 12위
    '시술_대분류',            # 13위
    '배아_진행_단계',          # 14위
    '수집된 신선 난자 수',      # 15위
    '총시술_bin3',           # 16위
    'BLASTOCYST_포함',       # 17위
    '배란 유도 유형',          # 18위
    '불임_원인_개수',          # 19위
    '총 생성 배아 수',         # 20위
    '시술횟수×나이',          # 파생 (교호작용)
]

print(f"\n선택된 Feature: {len(TOP_20_FEATURES)}개")
for i, feat in enumerate(TOP_20_FEATURES, 1):
    print(f"  {i:2d}. {feat}")

# Feature 존재 확인
available_features = []
for feat in TOP_20_FEATURES:
    if feat in train_df.columns:
        available_features.append(feat)
    else:
        print(f"⚠️  '{feat}' not found in data")

print(f"\n 실제 사용 Feature: {len(available_features)}개")

# ============================================================
# 3) 데이터 준비 (Top 20만!)
# ============================================================
train_data = train_df[available_features + [TARGET_COL]].copy()
test_data = test_df[available_features].copy()

print(f"\n최종 데이터:")
print(f"   Train: {train_data.shape}")
print(f"   Test: {test_data.shape}")
print(f"\nTarget 분포:")
print(train_data[TARGET_COL].value_counts(normalize=True))

# ============================================================
# 4) AutoGluon 학습
# ============================================================
print("\n" + "="*80)
print("🚀 AutoGluon Training - Top 20 Features Only!")
print("="*80)
print("\n⏰ 예상 소요 시간: 3시간")
print("💾 메모리 사용량: 4-8GB")
print("\n학습 시작...\n")

predictor = TabularPredictor(
    label=TARGET_COL,
    eval_metric='roc_auc',
    problem_type='binary',
    path='../outputs/autogluon_03.1_top20'
)

predictor.fit(
    train_data=train_data,
    time_limit=10800,        # 3시간
    presets='best_quality',
    num_bag_folds=10,        # 안정성
    num_bag_sets=2,          # 다양성
    num_stack_levels=2,      # 깊은 스태킹
    hyperparameters='multimodal',
    auto_stack=True,
    verbosity=2
)

# ============================================================
# 5) 결과 확인
# ============================================================
print("\n" + "="*80)
print("📊 Results")
print("="*80)

leaderboard = predictor.leaderboard(train_data, silent=True)
print("\n=== Model Leaderboard (Top 20) ===")
print(leaderboard.head(20).to_string())

best_model = leaderboard.iloc[0]['model']
best_score = leaderboard.iloc[0]['score_val']

print(f"\n Best Model: {best_model}")
print(f"🎯 Best CV AUC: {best_score:.6f}")

print(f"\n📊 성적표:")
print(f"   03번 (85개 Feature): 0.7407")
print(f"   03.1번 (20개 Feature): {best_score:.6f}")
print(f"   Improvement:        +{best_score - 0.7407:.6f}")

if best_score >= 0.7450:
    print("\n목표 달성! 1등 가능성 매우 높음! 🎉🎉🎉")
elif best_score >= 0.7425:
    print("\n대성공! 상위권 확정!")
elif best_score >= 0.7415:
    print("\n좋은 성적! 제출 추천!")
elif best_score > 0.7407:
    print("\n03번보다 높음! 제출 고려!")
else:
    print("\n03번이 더 나음... Feature Selection 효과 없음")

# ============================================================
# 6) 예측 및 Submission
# ============================================================
print("\n" + "="*80)
print("Predictions")
print("="*80)

test_predictions = predictor.predict_proba(test_data)

if isinstance(test_predictions, pd.DataFrame):
    test_proba = test_predictions[1].values
else:
    test_proba = test_predictions

print(f"\n예측값 통계:")
print(f"Min:  {test_proba.min():.6f}")
print(f"Max:  {test_proba.max():.6f}")
print(f"Mean: {test_proba.mean():.6f}")
print(f"Std:  {test_proba.std():.6f}")

# Submission
sub['probability'] = test_proba
output_file = f"../outputs/03.1_submission_top20_CV{best_score:.6f}.csv"
sub.to_csv(output_file, index=False)

print(f"\nSubmission saved!")
print(f"   File: {output_file}")

# ============================================================
# 7) 상세 정보 저장
# ============================================================
leaderboard.to_csv("../outputs/03.1_leaderboard.csv", index=False)
print("Leaderboard saved: ../outputs/03.1_leaderboard.csv")

try:
    importance = predictor.feature_importance(train_data)
    importance.to_csv("../outputs/03.1_feature_importance.csv")
    print("Feature importance saved")
    print("\n=== Top 20 Feature Importance (재확인) ===")
    print(importance.head(20).to_string())
except:
    print("Feature importance not available")

# ============================================================
# 8) 요약
# ============================================================
summary = f"""
{'='*80}
03.1번 - Top 20 Feature Selection 결과
{'='*80}

전략: 03번 Feature Importance Top 20만 선택
  - 85개 Feature → 20개 Feature (65개 제거!)
  - 노이즈 제거 + 핵심 Feature 집중

Best Model: {best_model}
CV AUC: {best_score:.6f}

성적 비교:
  03번 (85개):     0.7407 (LB: 0.742)
  03.1번 (20개):     {best_score:.6f}
  
Improvement: +{best_score - 0.7407:.6f}

예상 LB (03번 기준 +0.0013):
  03.1번: {best_score + 0.0013:.6f}

{'='*80}
현재 순위:
  1등: 0.74209
  나:  0.74200 (5등)
  
03.1번 예상 순위:
"""

if best_score + 0.0013 >= 0.74209:
    summary += "  (예상 LB: {:.5f})\n".format(best_score + 0.0013)
else:
    summary += "  상위권 (예상 LB: {:.5f})\n".format(best_score + 0.0013)

summary += "="*80

print("\n" + summary)

with open("../outputs/03.1_summary.txt", "w", encoding='utf-8') as f:
    f.write(summary)

print("\nSummary saved: ../outputs/03.1_summary.txt")
print("\n" + "="*80)
print("03.1번 COMPLETE!")
print("="*80)

03.1 - Top 20 Feature Selection + AutoGluon

전략: 03번 Feature Importance Top 20만 사용

Train shape: (256351, 69)
Test shape: (90067, 68)

전처리 중...
✅ 전처리 완료
   Train: (256351, 82)
   Test: (90067, 81)

🎯 Top 20 Feature Selection (03번 Feature Importance 기반)

선택된 Feature: 20개
   1. 이식된 배아 수
   2. 시술 당시 나이
   3. 나이_3구간
   4. 배아_저장_비율
   5. 이식배아_구간
   6. 나이×Day5
   7. Day5_이식_여부
   8. 배아_생성_효율
   9. 시술 시기 코드
  10. 저장된 배아 수
  11. 배아_이식_비율
  12. 시술_대분류
  13. 배아_진행_단계
  14. 수집된 신선 난자 수
  15. 총시술_bin3
  16. BLASTOCYST_포함
  17. 배란 유도 유형
  18. 불임_원인_개수
  19. 총 생성 배아 수
  20. 시술횟수×나이

 실제 사용 Feature: 20개


Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=10, num_bag_sets=2
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 10800 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: ../outputs/autogluon_03.1_top20/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.



최종 데이터:
   Train: (256351, 21)
   Test: (90067, 20)

Target 분포:
임신 성공 여부
0    0.741651
1    0.258349
Name: proportion, dtype: float64

🚀 AutoGluon Training - Top 20 Features Only!

⏰ 예상 소요 시간: 3시간
💾 메모리 사용량: 4-8GB

학습 시작...



Spend 2542 seconds for the sub-fit(s) during dynamic stacking.
Time left for full fit of AutoGluon: 8258 seconds.
Starting full fit now with num_stack_levels 0.
Beginning AutoGluon training ... Time limit = 8258s
AutoGluon will save models to "../outputs/autogluon_03.1_top20"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.6.0: Mon Jul 14 11:30:29 PDT 2025; root:xnu-11417.140.69~1/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       6.08 GB / 16.00 GB (38.0%)
Disk Space Avail:   206.83 GB / 460.43 GB (44.9%)
Train Data Rows:    256351
Train Data Columns: 20
Label Column:       임신 성공 여부
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                  


📊 Results

=== Model Leaderboard (Top 20) ===
                   model  score_test  score_val eval_metric  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   LightGBMLarge_BAG_L1    0.758143   0.733417     roc_auc       11.113005       9.522752    16.913163                11.113005                9.522752          16.913163            1       True          6
1        LightGBM_BAG_L1    0.741519   0.734102     roc_auc        6.065471       4.795352    10.874730                 6.065471                4.795352          10.874730            1       True          1
2    WeightedEnsemble_L2    0.740812   0.734957     roc_auc       51.447638      21.243602  2287.031656                 0.004058                0.039310           6.194374            2       True          7
3         XGBoost_BAG_L1    0.740777   0.734054     roc_auc       11.947266       3.164918   194.004320                11.947

Computing feature importance via permutation shuffling for 20 features using 5000 rows with 5 shuffle sets...



예측값 통계:
Min:  0.000273
Max:  0.635399
Mean: 0.259804
Std:  0.156654

Submission saved!
   File: ../outputs/03.1_submission_top20_CV0.733417.csv
Leaderboard saved: ../outputs/03.1_leaderboard.csv


	164.05s	= Expected runtime (32.81s per shuffle set)
	112.74s	= Actual runtime (Completed 5 of 5 shuffle sets)


Feature importance saved

=== Top 20 Feature Importance (재확인) ===
               importance    stddev   p_value  n  p99_high   p99_low
이식된 배아 수         0.017270  0.004218  0.000395  5  0.025955  0.008586
배아_이식_비율         0.015024  0.003372  0.000285  5  0.021966  0.008082
나이_3구간           0.011237  0.004490  0.002502  5  0.020482  0.001992
시술 당시 나이         0.007882  0.003204  0.002662  5  0.014479  0.001285
배아_생성_효율         0.007523  0.001626  0.000246  5  0.010870  0.004175
총 생성 배아 수        0.006957  0.003600  0.006219  5  0.014369 -0.000456
배아_진행_단계         0.006806  0.002230  0.001206  5  0.011398  0.002214
나이×Day5          0.006779  0.002646  0.002299  5  0.012228  0.001331
배란 유도 유형         0.006646  0.001128  0.000096  5  0.008969  0.004322
배아_저장_비율         0.005882  0.002896  0.005240  5  0.011844 -0.000080
이식배아_구간          0.005660  0.002296  0.002643  5  0.010388  0.000932
Day5_이식_여부       0.005457  0.001879  0.001450  5  0.009326  0.001588
시술 시기 코드         0.005007  0.000574  

### 03.2 하이퍼파라미터튜닝

In [4]:
import os
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# 0) Load
# ============================================================
TRAIN_PATH = "../data/train.csv"
TEST_PATH  = "../data/test.csv"
SUB_PATH   = "../data/sample_submission.csv"

train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sub   = pd.read_csv(SUB_PATH)

TARGET_COL = "임신 성공 여부"
ID_COL = "ID"

print(f"\nTrain shape: {train.shape}")
print(f"Test shape: {test.shape}")

# ============================================================
# 1) Feature Engineering (03번과 완전 동일!)
# ============================================================

# 고결측 변수 제거
HIGH_MISSING_COLS = [
    '난자 해동 경과일',           # 99.4%
    'PGS 시술 여부',             # 99.2%
    'PGD 시술 여부',             # 99.1%
    '착상 전 유전 검사 사용 여부', # 98.9%
    '임신 시도 또는 마지막 임신 경과 연수',  # 96.3%
]

EMBRYO_STAGE_COLS = [
    "단일 배아 이식 여부","착상 전 유전 진단 사용 여부","배아 생성 주요 이유",
    "총 생성 배아 수","미세주입된 난자 수","미세주입에서 생성된 배아 수",
    "이식된 배아 수","미세주입 배아 이식 수","저장된 배아 수",
    "미세주입 후 저장된 배아 수","해동된 배아 수","해동 난자 수",
    "수집된 신선 난자 수","저장된 신선 난자 수","혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수","기증자 정자와 혼합된 난자 수",
    "동결 배아 사용 여부","신선 배아 사용 여부","기증 배아 사용 여부","대리모 여부",
]

INFERTILITY_COLS = [
    "남성 주 불임 원인","남성 부 불임 원인","여성 주 불임 원인","여성 부 불임 원인",
    "부부 주 불임 원인","부부 부 불임 원인","불명확 불임 원인",
    "불임 원인 - 난관 질환","불임 원인 - 남성 요인","불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인","불임 원인 - 자궁경부 문제","불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도","불임 원인 - 정자 면역학적 요인","불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태"
]

def preprocess(df):
    df = df.copy()
    
    # 고결측 제거
    df = df.drop(columns=HIGH_MISSING_COLS, errors='ignore')
    
    # 시술_대분류
    def major_procedure(x):
        if pd.isna(x): return "Unknown"
        x = str(x)
        if "IUI" in x: return "IUI"
        if "ICSI" in x: return "ICSI"
        if "IVF" in x: return "IVF"
        if "DI" in x: return "DI"
        return "Other"
    
    df["시술_대분류"] = df["특정 시술 유형"].apply(major_procedure)
    
    # BLASTOCYST_포함
    s = df["특정 시술 유형"].astype("object").fillna("Unknown").astype(str)
    df["BLASTOCYST_포함"] = s.str.contains("BLASTOCYST", na=False).astype(int)
    
    # 배아_이식_여부
    df["배아_이식_미도달"] = df[EMBRYO_STAGE_COLS].isna().all(axis=1).astype(int)
    df["배아_이식_여부"] = 1 - df["배아_이식_미도달"]
    
    # 배아_진행_단계
    def embryo_stage(row):
        if row['배아_이식_여부'] == 0:
            return '배아단계_미도달'
        elif pd.isna(row['총 생성 배아 수']) or row['총 생성 배아 수'] == 0:
            return '배아생성_실패'
        elif pd.isna(row['이식된 배아 수']) or row['이식된 배아 수'] == 0:
            return '이식_미실시'
        else:
            return '이식_완료'
    
    df['배아_진행_단계'] = df.apply(embryo_stage, axis=1)
    
    # 총시술_bin3
    def collapse_trials(x):
        if x == '0회': return '0회'
        elif x in ['1회', '2회']: return '1-2회'
        else: return '3회 이상'
    
    df["총시술_bin3"] = df["총 시술 횟수"].apply(collapse_trials)
    
    # 나이_3구간
    def age_group_simple(age):
        if age == '알 수 없음': return 'Unknown'
        elif age == '만18-34세': return '34세 이하'
        elif age in ['만35-37세', '만38-39세']: return '35-39세'
        else: return '40세 이상'
    
    df['나이_3구간'] = df['시술 당시 나이'].apply(age_group_simple)
    
    # 이식배아_구간
    def embryo_count_bin(count):
        if pd.isna(count) or count == 0: return '0개'
        elif count <= 2: return '1-2개'
        else: return '3개 이상'
    
    df['이식배아_구간'] = df['이식된 배아 수'].apply(embryo_count_bin)
    
    # Day5_이식_여부
    df['Day5_이식_여부'] = (df['배아 이식 경과일'] == 5.0).astype(int)
    
    # 불임원인_복잡도
    df["불임_원인_개수"] = df[INFERTILITY_COLS].sum(axis=1)
    
    def infertility_complexity(count):
        if count == 0: return 'None'
        elif count == 1: return 'Single'
        elif count == 2: return 'Double'
        else: return 'Multiple'
    
    df['불임원인_복잡도'] = df['불임_원인_개수'].apply(infertility_complexity)
    
    # 배아_해동_실시_여부
    df['배아_해동_실시_여부'] = df['배아 해동 경과일'].notna().astype(int)
    
    # 비율 변수
    df['배아_생성_효율'] = df['총 생성 배아 수'] / (df['수집된 신선 난자 수'] + 1)
    df['배아_이식_비율'] = df['이식된 배아 수'] / (df['총 생성 배아 수'] + 1)
    df['배아_저장_비율'] = df['저장된 배아 수'] / (df['총 생성 배아 수'] + 1)
    
    # 교호작용
    df['나이×Day5'] = df['시술 당시 나이'].astype(str) + '_' + df['Day5_이식_여부'].astype(str)
    df['시술횟수×나이'] = df['총시술_bin3'] + '_' + df['나이_3구간']
    
    return df

print("\n전처리 중...")
train_df = preprocess(train)
test_df = preprocess(test)

print(f"✅ 전처리 완료 (03번과 동일)")
print(f"   Train: {train_df.shape}")
print(f"   Test: {test_df.shape}")

# ============================================================
# 2) 데이터 준비
# ============================================================
train_data = train_df.drop(columns=[ID_COL, '배아_이식_미도달'])
test_data = test_df.drop(columns=[ID_COL, '배아_이식_미도달'])

print(f"\n최종 Feature 개수: {len(train_data.columns) - 1}개")
print(f"\nTarget 분포:")
print(train_data[TARGET_COL].value_counts(normalize=True))

# ============================================================
# 🔥 3) AutoGluon 최대 강화! 🔥
# ============================================================
print("\n" + "="*80)
print("🚀 AutoGluon MAX POWER Training!")
print("="*80)
print("\n⏰ 소요 시간: 6시간 (03번의 3배!)")
print("💾 메모리: 8-16GB")
print("🎯 목표 CV: 0.745+")
print("\n강화 포인트:")
print("  1. 시간: 2시간 → 6시간 (3배)")
print("  2. Bagging: folds 5→10, sets 1→3")
print("  3. Stacking: levels 1→3 (깊은 앙상블)")
print("  4. 하이퍼파라미터: 각 모델 최대 강화")
print("\n학습 시작...\n")

predictor = TabularPredictor(
    label=TARGET_COL,
    eval_metric='roc_auc',
    problem_type='binary',
    path='../outputs/autogluon_10_hypermax'
)

# 🔥 하이퍼파라미터 최대 강화
hyperparameters = {
    'GBM': [
        {'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}},
        {},
        'GBMLarge',
    ],
    'CAT': {},
    'XGB': {},
    'FASTAI': {},
    'NN_TORCH': {},
    'RF': [
        {'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}},
        {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}},
    ],
}

predictor.fit(
    train_data=train_data,
    time_limit=21600,        # 🔥 6시간! (기존 2시간)
    presets='best_quality',
    num_bag_folds=10,        # 🔥 5→10 (더 안정적)
    num_bag_sets=3,          # 🔥 1→3 (더 다양)
    num_stack_levels=3,      # 🔥 1→3 (더 깊은 스태킹)
    hyperparameters=hyperparameters,
    auto_stack=True,
    verbosity=2
)

# ============================================================
# 4) 결과 확인
# ============================================================
print("\n" + "="*80)
print("📊 Results")
print("="*80)

leaderboard = predictor.leaderboard(train_data, silent=True)
print("\n=== Model Leaderboard (Top 30) ===")
print(leaderboard.head(30).to_string())

best_model = leaderboard.iloc[0]['model']
best_score = leaderboard.iloc[0]['score_val']

print(f"\n🏆 Best Model: {best_model}")
print(f"🎯 Best CV AUC: {best_score:.6f}")

print(f"\n📊 성적표:")
print(f"   03번 (2시간):  0.7407 (LB: 0.742)")
print(f"   10번 (6시간):  {best_score:.6f}")
print(f"   Improvement:  +{best_score - 0.7407:.6f}")

# 예상 LB
expected_lb = best_score + 0.0013  # 03번 기준
print(f"\n🎯 예상 Public LB: {expected_lb:.6f}")
print(f"   (03번은 CV+0.0013 상승)")

if best_score >= 0.7450:
    print("\n🎉🎉🎉 대박! 1등 거의 확정! 🎉🎉🎉")
    print(f"   예상 LB {expected_lb:.5f} > 현재 1등 0.74209")
elif best_score >= 0.7425:
    print("\n😊😊 대성공! 상위권 확정!")
elif best_score >= 0.7415:
    print("\n✅ 좋은 성적! 제출 강력 추천!")
elif best_score > 0.7407:
    print("\n👍 03번보다 높음! 제출 고려!")
else:
    print("\n🤔 03번과 비슷... 그래도 제출 가치 있음")

# ============================================================
# 5) Feature Importance
# ============================================================
print("\n" + "="*80)
print("Feature Importance")
print("="*80)

try:
    importance = predictor.feature_importance(train_data)
    importance.to_csv("../outputs/03.2_feature_importance.csv")
    print("✅ Feature importance saved")
    print("\n=== Top 30 Features ===")
    print(importance.head(30).to_string())
except:
    print("⚠️ Feature importance not available")

# ============================================================
# 6) 예측 및 Submission
# ============================================================
print("\n" + "="*80)
print("🎯 Predictions")
print("="*80)

test_predictions = predictor.predict_proba(test_data)

if isinstance(test_predictions, pd.DataFrame):
    test_proba = test_predictions[1].values
else:
    test_proba = test_predictions

print(f"\n예측값 통계:")
print(f"Min:  {test_proba.min():.6f}")
print(f"Max:  {test_proba.max():.6f}")
print(f"Mean: {test_proba.mean():.6f}")
print(f"Std:  {test_proba.std():.6f}")

# Submission
sub['probability'] = test_proba
output_file = f"../outputs/03.2_submission_hypermax_CV{best_score:.6f}.csv"
sub.to_csv(output_file, index=False)

print(f"\n✅ Submission saved!")
print(f"   File: {output_file}")

# ============================================================
# 7) 상세 정보 저장
# ============================================================
leaderboard.to_csv("../outputs/03.2_leaderboard.csv", index=False)
print("✅ Leaderboard saved: ../outputs/0.32_leaderboard.csv")

# ============================================================
# 8) 최종 요약
# ============================================================
summary = f"""
{'='*80}
🔥 03.2번 - 하이퍼파라미터 MAX 튜닝 결과
{'='*80}

전략: 03번 Feature (검증됨) + 시간 3배 + 설정 최대 강화

하이퍼파라미터 강화:
  - 시간: 2시간 → 6시간 (3배)
  - Bagging: folds 5→10, sets 1→3
  - Stacking: levels 1→3
  - 모델: GBM, CAT, XGB, NN, RF 전부 강화

Best Model: {best_model}
CV AUC: {best_score:.6f}

성적 비교:
  03번 (2시간):    0.7407 (LB: 0.7420)
  10번 (6시간):    {best_score:.6f}
  
Improvement: +{best_score - 0.7407:.6f}

예상 Public LB: {expected_lb:.6f}
  (03번 기준 CV + 0.0013)

{'='*80}
현재 순위:
  1등: 0.74209
  나:  0.74200 (5등)
  
10번 예상:
"""

if expected_lb >= 0.74209:
    summary += f"  🏆 1등 예상! (LB {expected_lb:.5f})\n"
elif expected_lb >= 0.74150:
    summary += f"  🥈 상위권 예상 (LB {expected_lb:.5f})\n"
else:
    summary += f"  상위권 유지 (LB {expected_lb:.5f})\n"

summary += f"""
{'='*80}
총 모델 수: {len(leaderboard)}

Top 5 Models:
"""

for i in range(min(5, len(leaderboard))):
    summary += f"  {i+1}. {leaderboard.iloc[i]['model']}: {leaderboard.iloc[i]['score_val']:.6f}\n"

summary += "="*80

print("\n" + summary)

with open("../outputs/03.2_summary.txt", "w", encoding='utf-8') as f:
    f.write(summary)

print("\n✅ Summary saved: ../outputs/10_summary.txt")
print("\n" + "="*80)
print("🔥🔥🔥 10번 COMPLETE! 🔥🔥🔥")
print("="*80)
print("\n최종 체크:")
print(f"  ✅ CV AUC: {best_score:.6f}")
print(f"  ✅ 예상 LB: {expected_lb:.6f}")
print(f"  ✅ Submission: {output_file}")
print("\n→ 바로 제출하세요! 1등 가능성 높음! 🏆")
print("="*80)


Train shape: (256351, 69)
Test shape: (90067, 68)

전처리 중...


Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=10, num_bag_sets=3
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 21600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: ../outputs/autogluon_10_hypermax/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.


✅ 전처리 완료 (03번과 동일)
   Train: (256351, 81)
   Test: (90067, 80)

최종 Feature 개수: 78개

Target 분포:
임신 성공 여부
0    0.741651
1    0.258349
Name: proportion, dtype: float64

🚀 AutoGluon MAX POWER Training!

⏰ 소요 시간: 6시간 (03번의 3배!)
💾 메모리: 8-16GB
🎯 목표 CV: 0.745+

강화 포인트:
  1. 시간: 2시간 → 6시간 (3배)
  2. Bagging: folds 5→10, sets 1→3
  3. Stacking: levels 1→3 (깊은 앙상블)
  4. 하이퍼파라미터: 각 모델 최대 강화

학습 시작...



Spend 5453 seconds for the sub-fit(s) during dynamic stacking.
Time left for full fit of AutoGluon: 16147 seconds.
Starting full fit now with num_stack_levels 0.
Beginning AutoGluon training ... Time limit = 16147s
AutoGluon will save models to "../outputs/autogluon_10_hypermax"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.6.0: Mon Jul 14 11:30:29 PDT 2025; root:xnu-11417.140.69~1/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       6.18 GB / 16.00 GB (38.6%)
Disk Space Avail:   210.92 GB / 460.43 GB (45.8%)
Train Data Rows:    256351
Train Data Columns: 78
Label Column:       임신 성공 여부
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:               


📊 Results


These features in provided data are not utilized by the predictor and will be ignored: ['불임 원인 - 여성 요인', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아_이식_여부']
Computing feature importance via permutation shuffling for 72 features using 5000 rows with 5 shuffle sets...



=== Model Leaderboard (Top 30) ===
                     model  score_test  score_val eval_metric  pred_time_test  pred_time_val      fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  RandomForestEntr_BAG_L1    0.931375   0.730583     roc_auc        1.791452       7.444626     11.861585                 1.791452                7.444626          11.861585            1       True          4
1  RandomForestGini_BAG_L1    0.929479   0.729776     roc_auc        1.701691       7.265590     11.198239                 1.701691                7.265590          11.198239            1       True          3
2     LightGBMLarge_BAG_L1    0.772744   0.739109     roc_auc       22.403445      18.864785     38.406015                22.403445               18.864785          38.406015            1       True          9
3      WeightedEnsemble_L2    0.760503   0.740810     roc_auc      168.033392      51.802508  11471.865762                 0

	2043.38s	= Expected runtime (408.68s per shuffle set)
	1196.76s	= Actual runtime (Completed 5 of 5 shuffle sets)


✅ Feature importance saved

=== Top 30 Features ===
                  importance    stddev       p_value  n  p99_high   p99_low
이식된 배아 수            0.023687  0.004292  1.238673e-04  5  0.032523  0.014850
시술 당시 나이            0.011249  0.003121  6.432862e-04  5  0.017674  0.004824
나이_3구간              0.009877  0.003298  1.293740e-03  5  0.016668  0.003086
배아_이식_비율            0.009632  0.002278  3.487698e-04  5  0.014322  0.004943
배아 이식 경과일           0.008219  0.002099  4.688847e-04  5  0.012540  0.003897
배아_저장_비율            0.007001  0.002539  1.755241e-03  5  0.012228  0.001774
총 생성 배아 수           0.006774  0.002622  2.229175e-03  5  0.012173  0.001376
나이×Day5             0.006565  0.001717  5.136822e-04  5  0.010100  0.003030
시술 시기 코드            0.006348  0.000434  2.602829e-06  5  0.007241  0.005454
배아_생성_효율            0.006009  0.000752  2.879392e-05  5  0.007556  0.004461
수집된 신선 난자 수         0.005068  0.000696  4.154320e-05  5  0.006500  0.003635
Day5_이식_여부          0.003882  0.0011

In [7]:
# ============================================================
# 03.2번 결과 수정 - WeightedEnsemble_L2로 재제출
# ============================================================
import pandas as pd
from autogluon.tabular import TabularPredictor

print("="*80)
print("03.2번 수정 - WeightedEnsemble_L2로 재제출")
print("="*80)

# 1. 저장된 predictor 로드
predictor = TabularPredictor.load('../outputs/autogluon_10_hypermax/')

# 2. Leaderboard score_val로 정렬
leaderboard = predictor.leaderboard(silent=True)

print("\n=== Leaderboard 컬럼 확인 ===")
print(leaderboard.columns.tolist())

# score_val로 정렬
leaderboard_sorted = leaderboard.sort_values('score_val', ascending=False)

print("\n=== Top 10 Models (score_val 기준) ===")
print(leaderboard_sorted.head(10)[['model', 'score_val']])

# 3. 진짜 Best Model
best_model = leaderboard_sorted.iloc[0]['model']
best_score = leaderboard_sorted.iloc[0]['score_val']

print(f"\n진짜 Best Model: {best_model}")
print(f"진짜 CV AUC: {best_score:.6f}")
print(f"\n성적표:")
print(f"   03번: 0.7407")
print(f"   10번: {best_score:.6f}")
print(f"   Improvement: +{best_score - 0.7407:.6f}")

# 4. Test 데이터 로드 및 전처리
test_df = pd.read_csv("../data/test.csv")

# 전처리 (03번과 동일)
def preprocess(df):
    df = df.copy()
    
    # 고결측 제거
    HIGH_MISSING_COLS = [
        '난자 해동 경과일', 'PGS 시술 여부', 'PGD 시술 여부',
        '착상 전 유전 검사 사용 여부', '임신 시도 또는 마지막 임신 경과 연수'
    ]
    df = df.drop(columns=HIGH_MISSING_COLS, errors='ignore')
    
    EMBRYO_STAGE_COLS = [
        "단일 배아 이식 여부","착상 전 유전 진단 사용 여부","배아 생성 주요 이유",
        "총 생성 배아 수","미세주입된 난자 수","미세주입에서 생성된 배아 수",
        "이식된 배아 수","미세주입 배아 이식 수","저장된 배아 수",
        "미세주입 후 저장된 배아 수","해동된 배아 수","해동 난자 수",
        "수집된 신선 난자 수","저장된 신선 난자 수","혼합된 난자 수",
        "파트너 정자와 혼합된 난자 수","기증자 정자와 혼합된 난자 수",
        "동결 배아 사용 여부","신선 배아 사용 여부","기증 배아 사용 여부","대리모 여부",
    ]
    
    INFERTILITY_COLS = [
        "남성 주 불임 원인","남성 부 불임 원인","여성 주 불임 원인","여성 부 불임 원인",
        "부부 주 불임 원인","부부 부 불임 원인","불명확 불임 원인",
        "불임 원인 - 난관 질환","불임 원인 - 남성 요인","불임 원인 - 배란 장애",
        "불임 원인 - 여성 요인","불임 원인 - 자궁경부 문제","불임 원인 - 자궁내막증",
        "불임 원인 - 정자 농도","불임 원인 - 정자 면역학적 요인","불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태"
    ]
    
    def major_procedure(x):
        if pd.isna(x): return "Unknown"
        x = str(x)
        if "IUI" in x: return "IUI"
        if "ICSI" in x: return "ICSI"
        if "IVF" in x: return "IVF"
        if "DI" in x: return "DI"
        return "Other"
    
    df["시술_대분류"] = df["특정 시술 유형"].apply(major_procedure)
    
    s = df["특정 시술 유형"].astype("object").fillna("Unknown").astype(str)
    df["BLASTOCYST_포함"] = s.str.contains("BLASTOCYST", na=False).astype(int)
    
    df["배아_이식_미도달"] = df[EMBRYO_STAGE_COLS].isna().all(axis=1).astype(int)
    df["배아_이식_여부"] = 1 - df["배아_이식_미도달"]
    
    def embryo_stage(row):
        if row['배아_이식_여부'] == 0:
            return '배아단계_미도달'
        elif pd.isna(row.get('총 생성 배아 수')) or row.get('총 생성 배아 수') == 0:
            return '배아생성_실패'
        elif pd.isna(row.get('이식된 배아 수')) or row.get('이식된 배아 수') == 0:
            return '이식_미실시'
        else:
            return '이식_완료'
    
    df['배아_진행_단계'] = df.apply(embryo_stage, axis=1)
    
    def collapse_trials(x):
        if x == '0회': return '0회'
        elif x in ['1회', '2회']: return '1-2회'
        else: return '3회 이상'
    
    df["총시술_bin3"] = df["총 시술 횟수"].apply(collapse_trials)
    
    def age_group_simple(age):
        if age == '알 수 없음': return 'Unknown'
        elif age == '만18-34세': return '34세 이하'
        elif age in ['만35-37세', '만38-39세']: return '35-39세'
        else: return '40세 이상'
    
    df['나이_3구간'] = df['시술 당시 나이'].apply(age_group_simple)
    
    def embryo_count_bin(count):
        if pd.isna(count) or count == 0: return '0개'
        elif count <= 2: return '1-2개'
        else: return '3개 이상'
    
    df['이식배아_구간'] = df['이식된 배아 수'].apply(embryo_count_bin)
    df['Day5_이식_여부'] = (df['배아 이식 경과일'] == 5.0).astype(int)
    df["불임_원인_개수"] = df[INFERTILITY_COLS].sum(axis=1)
    
    def infertility_complexity(count):
        if count == 0: return 'None'
        elif count == 1: return 'Single'
        elif count == 2: return 'Double'
        else: return 'Multiple'
    
    df['불임원인_복잡도'] = df['불임_원인_개수'].apply(infertility_complexity)
    df['배아_해동_실시_여부'] = df['배아 해동 경과일'].notna().astype(int)
    df['배아_생성_효율'] = df['총 생성 배아 수'] / (df['수집된 신선 난자 수'] + 1)
    df['배아_이식_비율'] = df['이식된 배아 수'] / (df['총 생성 배아 수'] + 1)
    df['배아_저장_비율'] = df['저장된 배아 수'] / (df['총 생성 배아 수'] + 1)
    df['나이×Day5'] = df['시술 당시 나이'].astype(str) + '_' + df['Day5_이식_여부'].astype(str)
    df['시술횟수×나이'] = df['총시술_bin3'] + '_' + df['나이_3구간']
    
    return df

test_df = preprocess(test_df)
test_data = test_df.drop(columns=['ID', '배아_이식_미도달'])

# 5. 예측
print("\n예측 중...")
test_predictions = predictor.predict_proba(test_data, model=best_model)

if isinstance(test_predictions, pd.DataFrame):
    test_proba = test_predictions[1].values
else:
    test_proba = test_predictions

print(f"\n예측값 통계:")
print(f"Min:  {test_proba.min():.6f}")
print(f"Max:  {test_proba.max():.6f}")
print(f"Mean: {test_proba.mean():.6f}")

# 6. Submission
sub = pd.read_csv("../data/sample_submission.csv")
sub['probability'] = test_proba

output_file = f"../outputs/10_FIXED_submission_CV{best_score:.6f}.csv"
sub.to_csv(output_file, index=False)

print(f"\n✅ 수정된 Submission 저장!")
print(f"   Model: {best_model}")
print(f"   CV: {best_score:.6f}")
print(f"   File: {output_file}")

expected_lb = best_score + 0.0013
print(f"\n🎯 예상 LB: {expected_lb:.6f}")
print(f"   현재 1등: 0.74209")

if expected_lb >= 0.74209:
    print("\n🏆 1등 가능!")
else:
    print("\n😊 상위권!")

03.2번 수정 - WeightedEnsemble_L2로 재제출

=== Leaderboard 컬럼 확인 ===
['model', 'score_val', 'eval_metric', 'pred_time_val', 'fit_time', 'pred_time_val_marginal', 'fit_time_marginal', 'stack_level', 'can_infer', 'fit_order']

=== Top 10 Models (score_val 기준) ===
                     model  score_val
0      WeightedEnsemble_L2   0.740810
1          CatBoost_BAG_L1   0.740375
2        LightGBMXT_BAG_L1   0.740120
3          LightGBM_BAG_L1   0.739933
4           XGBoost_BAG_L1   0.739630
5   NeuralNetFastAI_BAG_L1   0.739541
6     LightGBMLarge_BAG_L1   0.739109
7    NeuralNetTorch_BAG_L1   0.738706
8  RandomForestEntr_BAG_L1   0.730583
9  RandomForestGini_BAG_L1   0.729776

진짜 Best Model: WeightedEnsemble_L2
진짜 CV AUC: 0.740810

성적표:
   03번: 0.7407
   10번: 0.740810
   Improvement: +0.000110

예측 중...

예측값 통계:
Min:  0.000210
Max:  0.731375
Mean: 0.258415

✅ 수정된 Submission 저장!
   Model: WeightedEnsemble_L2
   CV: 0.740810
   File: ../outputs/10_FIXED_submission_CV0.740810.csv

🎯 예상 LB: 0.742110
 

### 03.3 Seed 123으로 바꿔서 test

In [28]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("03.3번 - Random Seed 123 버전")
print("="*80)
print("전략: 03번 완전 동일 + Seed만 123으로 변경")
print("="*80)

# 1. 데이터 로드
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")
submission = pd.read_csv("../data/sample_submission.csv")

print(f"\nTrain shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

# 2. 전처리 함수 (03번과 완전 동일)
def preprocess(df):
    df_copy = df.copy()
    
    # 시술_대분류
    def major_procedure(x):
        if pd.isna(x):
            return "Unknown"
        if "IUI" in x:
            return "IUI"
        if "DI" in x:
            return "Other"
        if "ICSI" in x:
            return "ICSI"
        if "IVF" in x:
            return "IVF"
        return "Other"
    
    df_copy["시술_대분류"] = df_copy["특정 시술 유형"].apply(major_procedure)
    
    # BLASTOCYST 포함 여부
    df_copy["BLASTOCYST_포함"] = df_copy["특정 시술 유형"].str.contains("BLASTOCYST", na=False).astype(int)
    
    # 배아 이식 여부
    embryo_stage_cols = [
        "단일 배아 이식 여부", "착상 전 유전 진단 사용 여부", "배아 생성 주요 이유",
        "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수",
        "이식된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수",
        "미세주입 후 저장된 배아 수", "해동된 배아 수", "해동 난자 수",
        "수집된 신선 난자 수", "저장된 신선 난자 수", "혼합된 난자 수",
        "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수",
        "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부",
    ]
    
    df_copy["배아_이식_미도달"] = df_copy[embryo_stage_cols].isna().all(axis=1).astype(int)
    df_copy["배아_이식_여부"] = 1 - df_copy["배아_이식_미도달"]
    
    # 배아 진행 단계
    def embryo_stage(row):
        if row['배아_이식_여부'] == 0:
            return '배아단계_미도달'
        elif pd.isna(row['총 생성 배아 수']) or row['총 생성 배아 수'] == 0:
            return '배아생성_실패'
        elif pd.isna(row['이식된 배아 수']) or row['이식된 배아 수'] == 0:
            return '이식_미실시'
        else:
            return '이식_완료'
    
    df_copy['배아_진행_단계'] = df_copy.apply(embryo_stage, axis=1)
    
    # 총시술_bin3
    def collapse_trials(x):
        if x == '0회':
            return '0회'
        elif x in ['1회', '2회']:
            return '1-2회'
        else:
            return '3회 이상'
    
    df_copy["총시술_bin3"] = df_copy["총 시술 횟수"].apply(collapse_trials)
    
    # 나이_3구간
    def age_group_simple(age):
        if age == '알 수 없음':
            return 'Unknown'
        elif age == '만18-34세':
            return '34세 이하'
        elif age in ['만35-37세', '만38-39세']:
            return '35-39세'
        else:
            return '40세 이상'
    
    df_copy['나이_3구간'] = df_copy['시술 당시 나이'].apply(age_group_simple)
    
    # 이식배아_구간
    def embryo_count_bin(count):
        if pd.isna(count) or count == 0:
            return '0개'
        elif count <= 2:
            return '1-2개'
        else:
            return '3개 이상'
    
    df_copy['이식배아_구간'] = df_copy['이식된 배아 수'].apply(embryo_count_bin)
    
    # Day5_이식_여부
    df_copy['Day5_이식_여부'] = (df_copy['배아 이식 경과일'] == 5.0).astype(int)
    
    # 불임원인_복잡도
    infertility_cols = [
        "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인",
        "부부 주 불임 원인", "부부 부 불임 원인", "불명확 불임 원인",
        "불임 원인 - 난관 질환", "불임 원인 - 남성 요인", "불임 원인 - 배란 장애",
        "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
        "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태"
    ]
    
    df_copy["불임_원인_개수"] = df_copy[infertility_cols].sum(axis=1)
    
    def infertility_complexity(count):
        if count == 0:
            return 'None'
        elif count == 1:
            return 'Single'
        elif count == 2:
            return 'Double'
        else:
            return 'Multiple'
    
    df_copy['불임원인_복잡도'] = df_copy['불임_원인_개수'].apply(infertility_complexity)
    
    # 배아_해동_실시_여부
    df_copy['배아_해동_실시_여부'] = df_copy['배아 해동 경과일'].notna().astype(int)
    
    # 배아 효율 비율 변수들
    df_copy['배아_생성_효율'] = df_copy['총 생성 배아 수'] / (df_copy['수집된 신선 난자 수'] + 1)
    df_copy['배아_이식_비율'] = df_copy['이식된 배아 수'] / (df_copy['총 생성 배아 수'] + 1)
    df_copy['배아_저장_비율'] = df_copy['저장된 배아 수'] / (df_copy['총 생성 배아 수'] + 1)
    
    # 교호작용 변수들
    df_copy['나이×Day5'] = df_copy['시술 당시 나이'].astype(str) + '_' + df_copy['Day5_이식_여부'].astype(str)
    df_copy['시술횟수×나이'] = df_copy['총시술_bin3'] + '_' + df_copy['나이_3구간']
    
    return df_copy

# 3. 전처리 적용
print("\n전처리 중...")
train_df = preprocess(train_df)
test_df = preprocess(test_df)

print(f"전처리 후 Train shape: {train_df.shape}")
print(f"전처리 후 Test shape: {test_df.shape}")

# 4. ID 제거
train_data = train_df.drop(columns=['ID', '배아_이식_미도달'])
test_data = test_df.drop(columns=['ID', '배아_이식_미도달'])

print(f"\nFinal Train shape: {train_data.shape}")
print(f"Target distribution:\n{train_data['임신 성공 여부'].value_counts(normalize=True)}")

# ============================
# 5. AutoGluon 학습 (Seed 123)
# ============================
print("\n" + "="*80)
print("AutoGluon Training - Random Seed 123")
print("="*80)
print("\n예상 소요 시간: 2시간")
print("Random Seed: 123 (03번은 42)")
print("메모리 사용량: 4-8GB")
print("\n학습 시작...\n")

predictor = TabularPredictor(
    label='임신 성공 여부',
    eval_metric='roc_auc',
    problem_type='binary',
    path='../outputs/autogluon_seed123_v2'
)

predictor.fit(
    train_data=train_data,
    time_limit=7200,  # 2시간
    presets='best_quality',
    num_bag_folds=5,
    num_bag_sets=1,
    num_stack_levels=1,
    ag_args_fit={'num_gpus': 0},
    hyperparameters={
        'GBM': {'seed': 123},
        'CAT': {'random_seed': 123},
        'XGB': {'seed': 123},
        'NN_TORCH': {'seed': 123},
        'FASTAI': {'seed': 123},
        'RF': {'random_state': 123}
    },
    verbosity=2
)

# ============================
# 6. 결과 확인
# ============================
print("\n" + "="*80)
print("Results")
print("="*80)

# Leaderboard (score_val 기준 정렬!)
leaderboard = predictor.leaderboard(train_data, silent=True)
leaderboard_sorted = leaderboard.sort_values('score_val', ascending=False)

print("\n=== Model Leaderboard (Top 15, score_val 기준) ===")
print(leaderboard_sorted.head(15)[['model', 'score_val']].to_string())

# Best model
best_model = leaderboard_sorted.iloc[0]['model']
best_score = leaderboard_sorted.iloc[0]['score_val']

print(f"\n🏆 Best Model: {best_model}")
print(f"🎯 Best CV AUC: {best_score:.6f}")

# ============================
# 7. 예측 및 Submission
# ============================
print("\n" + "="*80)
print("Predictions")
print("="*80)

# Test 예측 (Best Model 사용!)
test_predictions = predictor.predict_proba(test_data, model=best_model)

if isinstance(test_predictions, pd.DataFrame):
    test_proba = test_predictions[1].values
else:
    test_proba = test_predictions

print(f"\n예측값 통계:")
print(f"Min:  {test_proba.min():.6f}")
print(f"Max:  {test_proba.max():.6f}")
print(f"Mean: {test_proba.mean():.6f}")

# Submission 생성
submission['probability'] = test_proba
output_file = f"../outputs/03.3_seed123_CV{best_score:.6f}.csv"
submission.to_csv(output_file, index=False)

print(f"\nSubmission saved!")
print(f"   File: {output_file}")
print(f"   Model: {best_model}")
print(f"   Seed: 123")

# ============================
# 8. 상세 정보 저장
# ============================
leaderboard_sorted.to_csv("../outputs/03.3_seed123_leaderboard.csv", index=False)
print("Leaderboard saved")

try:
    importance = predictor.feature_importance(train_data)
    importance.to_csv("../outputs/03.3_seed123_feature_importance.csv")
    print("Feature importance saved")
except:
    pass

# 요약
summary = f"""
{'='*80}
03.3번 - Random Seed 123 결과
{'='*80}

전략: 03번 완전 동일 + Random Seed만 123으로 변경

Best Model: {best_model}
CV AUC: {best_score:.6f}

성적 비교:
  03번 (Seed 42):  0.7407 (LB: 0.7420)
  11번 (Seed 123): {best_score:.6f}
  
차이: {best_score - 0.7407:+.6f}

예상 LB: {expected_lb:.6f}
현재 1등: 0.74209

{'='*80}
"""

print("\n" + summary)

with open("../outputs/03.3_seed123_summary.txt", "w") as f:
    f.write(summary)

print("Summary saved")
print("\n" + "="*80)
print("03.3번 (Seed 123) COMPLETE!")
print("="*80)

03.3번 - Random Seed 123 버전
전략: 03번 완전 동일 + Seed만 123으로 변경

Train shape: (256351, 69)
Test shape: (90067, 68)

전처리 중...


Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=5, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 7200 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: ../outputs/autogluon_seed123_v2/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.


전처리 후 Train shape: (256351, 86)
전처리 후 Test shape: (90067, 85)

Final Train shape: (256351, 84)
Target distribution:
임신 성공 여부
0    0.741651
1    0.258349
Name: proportion, dtype: float64

AutoGluon Training - Random Seed 123

예상 소요 시간: 2시간
Random Seed: 123 (03번은 42)
메모리 사용량: 4-8GB

학습 시작...



Spend 47 seconds for the sub-fit(s) during dynamic stacking.
Time left for full fit of AutoGluon: 7153 seconds.
Starting full fit now with num_stack_levels 1.
Beginning AutoGluon training ... Time limit = 7153s
AutoGluon will save models to "../outputs/autogluon_seed123_v2"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.6.0: Mon Jul 14 11:30:29 PDT 2025; root:xnu-11417.140.69~1/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       3.46 GB / 16.00 GB (21.6%)
Disk Space Avail:   202.38 GB / 460.43 GB (44.0%)
Train Data Rows:    256351
Train Data Columns: 83
Label Column:       임신 성공 여부
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    


Results

=== Model Leaderboard (Top 15, score_val 기준) ===
                 model  score_val
2  RandomForest_BAG_L2   0.743485
3  WeightedEnsemble_L3   0.743485
0  RandomForest_BAG_L1   0.729995
1  WeightedEnsemble_L2   0.729995

🏆 Best Model: RandomForest_BAG_L2
🎯 Best CV AUC: 0.743485

Predictions

예측값 통계:
Min:  0.000754
Max:  0.789891
Mean: 0.257188

Submission saved!
   File: ../outputs/03.3_seed123_CV0.743485.csv
   Model: RandomForest_BAG_L2
   Seed: 123
Leaderboard saved


These features in provided data are not utilized by the predictor and will be ignored: ['불임 원인 - 여성 요인', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아_이식_여부']
Computing feature importance via permutation shuffling for 77 features using 5000 rows with 5 shuffle sets...
	211.65s	= Expected runtime (42.33s per shuffle set)
	34.8s	= Actual runtime (Completed 5 of 5 shuffle sets)


Feature importance saved


03.3번 - Random Seed 123 결과

전략: 03번 완전 동일 + Random Seed만 123으로 변경

Best Model: RandomForest_BAG_L2
CV AUC: 0.743485

성적 비교:
  03번 (Seed 42):  0.7407 (LB: 0.7420)
  11번 (Seed 123): 0.743485

차이: +0.002785

예상 LB: 0.746178
현재 1등: 0.74209


Summary saved

03.3번 (Seed 123) COMPLETE!


### 03.4 num_bag_sets=3으로 지정

In [14]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("AutoGluon Experiment 03.4")
print("="*60)

# 1. 데이터 로드
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")
submission = pd.read_csv("../data/sample_submission.csv")

print(f"\nTrain shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

# 2. 전처리 함수 (기존과 동일)
def preprocess(df):
    df_copy = df.copy()
    
    # 시술_대분류
    def major_procedure(x):
        if pd.isna(x):
            return "Unknown"
        if "IUI" in x:
            return "IUI"
        if "DI" in x:
            return "Other"
        if "ICSI" in x:
            return "ICSI"
        if "IVF" in x:
            return "IVF"
        return "Other"
    
    df_copy["시술_대분류"] = df_copy["특정 시술 유형"].apply(major_procedure)
    
    # BLASTOCYST 포함 여부
    df_copy["BLASTOCYST_포함"] = df_copy["특정 시술 유형"].str.contains("BLASTOCYST", na=False).astype(int)
    
    # 배아 이식 여부
    embryo_stage_cols = [
        "단일 배아 이식 여부", "착상 전 유전 진단 사용 여부", "배아 생성 주요 이유",
        "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수",
        "이식된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수",
        "미세주입 후 저장된 배아 수", "해동된 배아 수", "해동 난자 수",
        "수집된 신선 난자 수", "저장된 신선 난자 수", "혼합된 난자 수",
        "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수",
        "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부",
    ]
    
    df_copy["배아_이식_미도달"] = df_copy[embryo_stage_cols].isna().all(axis=1).astype(int)
    df_copy["배아_이식_여부"] = 1 - df_copy["배아_이식_미도달"]
    
    # 배아 진행 단계
    def embryo_stage(row):
        if row['배아_이식_여부'] == 0:
            return '배아단계_미도달'
        elif pd.isna(row['총 생성 배아 수']) or row['총 생성 배아 수'] == 0:
            return '배아생성_실패'
        elif pd.isna(row['이식된 배아 수']) or row['이식된 배아 수'] == 0:
            return '이식_미실시'
        else:
            return '이식_완료'
    
    df_copy['배아_진행_단계'] = df_copy.apply(embryo_stage, axis=1)
    
    # 총시술_bin3
    def collapse_trials(x):
        if x == '0회':
            return '0회'
        elif x in ['1회', '2회']:
            return '1–2회'
        else:
            return '3회 이상'
    
    df_copy["총시술_bin3"] = df_copy["총 시술 횟수"].apply(collapse_trials)
    
    # 나이_3구간
    def age_group_simple(age):
        if age == '알 수 없음':
            return 'Unknown'
        elif age == '만18-34세':
            return '34세 이하'
        elif age in ['만35-37세', '만38-39세']:
            return '35-39세'
        else:
            return '40세 이상'
    
    df_copy['나이_3구간'] = df_copy['시술 당시 나이'].apply(age_group_simple)
    
    # 이식배아_구간
    def embryo_count_bin(count):
        if pd.isna(count) or count == 0:
            return '0개'
        elif count <= 2:
            return '1-2개'
        else:
            return '3개 이상'
    
    df_copy['이식배아_구간'] = df_copy['이식된 배아 수'].apply(embryo_count_bin)
    
    # Day5_이식_여부
    df_copy['Day5_이식_여부'] = (df_copy['배아 이식 경과일'] == 5.0).astype(int)
    
    # 불임원인_복잡도
    infertility_cols = [
        "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인",
        "부부 주 불임 원인", "부부 부 불임 원인", "불명확 불임 원인",
        "불임 원인 - 난관 질환", "불임 원인 - 남성 요인", "불임 원인 - 배란 장애",
        "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
        "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태"
    ]
    
    df_copy["불임_원인_개수"] = df_copy[infertility_cols].sum(axis=1)
    
    def infertility_complexity(count):
        if count == 0:
            return 'None'
        elif count == 1:
            return 'Single'
        elif count == 2:
            return 'Double'
        else:
            return 'Multiple'
    
    df_copy['불임원인_복잡도'] = df_copy['불임_원인_개수'].apply(infertility_complexity)
    
    # 배아_해동_실시_여부
    df_copy['배아_해동_실시_여부'] = df_copy['배아 해동 경과일'].notna().astype(int)
    
    # 배아 효율 비율 변수들
    df_copy['배아_생성_효율'] = df_copy['총 생성 배아 수'] / (df_copy['수집된 신선 난자 수'] + 1)
    df_copy['배아_이식_비율'] = df_copy['이식된 배아 수'] / (df_copy['총 생성 배아 수'] + 1)
    df_copy['배아_저장_비율'] = df_copy['저장된 배아 수'] / (df_copy['총 생성 배아 수'] + 1)
    
    # 교호작용 변수들
    df_copy['나이×Day5'] = df_copy['시술 당시 나이'].astype(str) + '_' + df_copy['Day5_이식_여부'].astype(str)
    df_copy['시술횟수×나이'] = df_copy['총시술_bin3'] + '_' + df_copy['나이_3구간']
    
    return df_copy

# 3. 전처리 적용
print("\n전처리 중...")
train_df = preprocess(train_df)
test_df = preprocess(test_df)

print(f"전처리 후 Train shape: {train_df.shape}")
print(f"전처리 후 Test shape: {test_df.shape}")

# 4. ID 제거 (AutoGluon은 자동으로 타겟 찾음)
train_data = train_df.drop(columns=['ID', '배아_이식_미도달'])
test_data = test_df.drop(columns=['ID', '배아_이식_미도달'])

print(f"\nFinal Train shape: {train_data.shape}")
print(f"Target distribution:\n{train_data['임신 성공 여부'].value_counts(normalize=True)}")

# ============================
# 5. AutoGluon 학습
# ============================
print("\n" + "="*60)
print("Starting AutoGluon Training")
print("="*60)
print("\n⏰ 예상 소요 시간: 2시간")
print("💾 메모리 사용량: 4-8GB 예상")
print("\n학습 시작...\n")

predictor = TabularPredictor(
    label='임신 성공 여부',
    eval_metric='roc_auc',
    problem_type='binary',
    path='../outputs/autogluon_models_03.4'
)

predictor.fit(
    train_data=train_data,
    time_limit=10800,  # 2시간
    presets='best_quality',
    num_bag_folds=5,
    num_bag_sets=3,
    num_stack_levels=1,
    verbosity=2
)

# ============================
# 6. 결과 확인
# ============================
print("\n" + "="*60)
print("AutoGluon Results")
print("="*60)

# Leaderboard
leaderboard = predictor.leaderboard(train_data, silent=True)
print("\n=== Model Leaderboard (Top 15) ===")
print(leaderboard.head(15).to_string())

# Best model
best_model = leaderboard.iloc[0]['model']
best_score = leaderboard.iloc[0]['score_val']
print(f"\nBest Model: {best_model}")
print(f"Best CV AUC: {best_score:.6f}")

# ============================
# 7. Feature Importance
# ============================
print("\n" + "="*60)
print("Feature Importance")
print("="*60)

try:
    importance = predictor.feature_importance(train_data)
    print("\n=== Top 20 Features ===")
    print(importance.head(20).to_string())
except:
    print("Feature importance not available for ensemble model")

# ============================
# 8. 예측 및 Submission
# ============================
print("\n" + "="*60)
print("Generating Predictions")
print("="*60)

# Test 예측
test_predictions = predictor.predict_proba(test_data)

# 확률값 추출 (클래스 1의 확률)
if isinstance(test_predictions, pd.DataFrame):
    test_proba = test_predictions[1].values
else:
    test_proba = test_predictions

print(f"\n예측값 통계:")
print(f"Min:  {test_proba.min():.6f}")
print(f"Max:  {test_proba.max():.6f}")
print(f"Mean: {test_proba.mean():.6f}")
print(f"Std:  {test_proba.std():.6f}")

# Submission 생성
submission['probability'] = test_proba
submission.to_csv("../outputs/03.4_submission_autogluon(0209).csv", index=False)

print("\nAutoGluon Submission saved!")
print(f"   File: ../outputs/03.4_submission_autogluon(0209).csv")

# ============================
# 9. 상세 정보 저장
# ============================
print("\n" + "="*60)
print("Saving Details")
print("="*60)

# Leaderboard 전체 저장
leaderboard.to_csv("../outputs/03.4_autogluon_leaderboard.csv", index=False)
print("✅ Leaderboard saved: ../outputs/03.4_autogluon_leaderboard.csv")

# Feature importance 저장
try:
    importance.to_csv("../outputs/03.4_autogluon_feature_importance.csv")
    print("✅ Feature importance saved: ../outputs/03.4_autogluon_feature_importance.csv")
except:
    pass

# ============================
# 10. 요약
# ============================
print("\n" + "="*60)
print("SUMMARY")
print("="*60)

summary = f"""
Best Model: {best_model}
CV AUC: {best_score:.6f}

Total Models Trained: {len(leaderboard)}
Ensemble Type: {predictor.model_best}

Top 3 Models:
"""

for i in range(min(3, len(leaderboard))):
    summary += f"  {i+1}. {leaderboard.iloc[i]['model']}: {leaderboard.iloc[i]['score_val']:.6f}\n"

print(summary)

# 요약 파일 저장
with open("../outputs/autogluon_summary.txt", "w") as f:
    f.write(summary)

print("\n✅ Summary saved: ../outputs/03.4_autogluon_summary.txt")
print("\n" + "="*60)
print("AutoGluon Experiment Complete! 🎉")
print("="*60)

AutoGluon Experiment 03.4

Train shape: (256351, 69)
Test shape: (90067, 68)

전처리 중...


Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=5, num_bag_sets=3
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 10800 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: ../outputs/autogluon_models_03.4/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.


전처리 후 Train shape: (256351, 86)
전처리 후 Test shape: (90067, 85)

Final Train shape: (256351, 84)
Target distribution:
임신 성공 여부
0    0.741651
1    0.258349
Name: proportion, dtype: float64

Starting AutoGluon Training

⏰ 예상 소요 시간: 2시간
💾 메모리 사용량: 4-8GB 예상

학습 시작...



Spend 2774 seconds for the sub-fit(s) during dynamic stacking.
Time left for full fit of AutoGluon: 8026 seconds.
Starting full fit now with num_stack_levels 0.
Beginning AutoGluon training ... Time limit = 8026s
AutoGluon will save models to "../outputs/autogluon_models_03.4"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.6.0: Mon Jul 14 11:30:29 PDT 2025; root:xnu-11417.140.69~1/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       4.72 GB / 16.00 GB (29.5%)
Disk Space Avail:   205.41 GB / 460.43 GB (44.6%)
Train Data Rows:    256351
Train Data Columns: 83
Label Column:       임신 성공 여부
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                 


AutoGluon Results


These features in provided data are not utilized by the predictor and will be ignored: ['불임 원인 - 여성 요인', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아_이식_여부']
Computing feature importance via permutation shuffling for 77 features using 5000 rows with 5 shuffle sets...



=== Model Leaderboard (Top 15) ===
                       model  score_test  score_val eval_metric  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   RandomForest_r195_BAG_L1    0.998909   0.703452     roc_auc        3.734834       7.134382    57.589181                 3.734834                7.134382          57.589181            1       True         26
1      ExtraTrees_r42_BAG_L1    0.958349   0.722289     roc_auc        2.676122       8.648223    62.250063                 2.676122                8.648223          62.250063            1       True         22
2    RandomForestEntr_BAG_L1    0.929971   0.730819     roc_auc        1.876779       7.104161    14.544785                 1.876779                7.104161          14.544785            1       True          6
3    RandomForestGini_BAG_L1    0.928349   0.730036     roc_auc        2.060276       8.269329    14.485444             

	1263.88s	= Expected runtime (252.78s per shuffle set)
	719.41s	= Actual runtime (Completed 5 of 5 shuffle sets)



=== Top 20 Features ===
                  importance    stddev       p_value  n  p99_high   p99_low
이식된 배아 수            0.021769  0.004232  1.630955e-04  5  0.030483  0.013055
시술 당시 나이            0.011291  0.003111  6.266278e-04  5  0.017696  0.004886
나이_3구간              0.010415  0.003554  1.401910e-03  5  0.017732  0.003098
배아_이식_비율            0.009373  0.002086  2.757492e-04  5  0.013667  0.005079
이식배아_구간             0.009241  0.003163  1.418034e-03  5  0.015753  0.002728
나이×Day5             0.007598  0.002226  7.911116e-04  5  0.012182  0.003015
배아_저장_비율            0.007538  0.002600  1.460510e-03  5  0.012892  0.002183
배아 이식 경과일           0.007324  0.001887  4.852665e-04  5  0.011210  0.003438
총 생성 배아 수           0.006645  0.002688  2.617837e-03  5  0.012180  0.001109
배아_생성_효율            0.006558  0.001146  1.073808e-04  5  0.008917  0.004199
시술 시기 코드            0.006548  0.000496  3.929399e-06  5  0.007569  0.005526
수집된 신선 난자 수         0.004736  0.000636  3.805669e-05  5  0.0060

03.4 결과는 사용 못할 것 같음. 